In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

# Project paths
PROJECT_ROOT = Path(r"C:\Users\sdnea\Dev\permit-data-dev")
BRONZE_PATH = PROJECT_ROOT / "bronze"
DB_PATH = PROJECT_ROOT / "permit_data.db"


In [2]:
# Connect (creates file if it doesn't exist)
conn = sqlite3.connect(DB_PATH)


In [3]:
def load_to_sqlite(file_path: Path, table_name: str, conn):
    """Load Excel or CSV into SQLite as a table."""
    if file_path.suffix.lower() == ".csv":
        df = pd.read_csv(file_path)
    elif file_path.suffix.lower() in [".xls", ".xlsx"]:
        df = pd.read_excel(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path.suffix}")
    
    df.to_sql(table_name, conn, if_exists="replace", index=False)
    print(f"✅ Loaded {file_path.name} → {table_name} ({len(df)} rows)")


In [6]:
files = {
    "permits_2023_bronze": BRONZE_PATH / "2023 Building Permits.csv",
    "permits_2024_bronze": BRONZE_PATH / "2024 Building Permits.csv",
    "permits_2025_bronze": BRONZE_PATH / "2025 Building Permits.csv",
}

for table, path in files.items():
    load_to_sqlite(path, table, conn)


✅ Loaded 2023 Building Permits.csv → permits_2023_bronze (503 rows)
✅ Loaded 2024 Building Permits.csv → permits_2024_bronze (1411 rows)
✅ Loaded 2025 Building Permits.csv → permits_2025_bronze (1124 rows)


In [7]:
for table in files.keys():
    rows = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {rows} rows")


permits_2023_bronze: 503 rows
permits_2024_bronze: 1411 rows
permits_2025_bronze: 1124 rows


In [8]:
conn.close()
